### Inception transfer learning

In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

train_data_path = 'data/train/'
validation_data_path = 'data/validation/'
image_size = (299, 299)

# base model
base_model = InceptionV3(weights='imagenet', include_top=False)
# create our model based on base model
x = base_model.output
# create global average pooling
x = GlobalAveragePooling2D()(x)
# create dropout layer
x = Dropout(0.5)(x)
# create last layer for classfication
predictions = Dense(2, activation='softmax')(x)
# create model
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

# lock base model layer，only train top layer
for layer in base_model.layers:
    layer.trainable = False

# compile model
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

# create data generator for data argumentation
gen = ImageDataGenerator()
train_generator = gen.flow_from_directory(train_data_path, image_size, shuffle=False, batch_size=16)
validation_generator = gen.flow_from_directory(validation_data_path, image_size, shuffle=False, batch_size=16)


#checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.inception.hdf5', verbose=1, save_best_only=True)
# fit model
#model.fit_generator(train_generator,        
                    steps_per_epoch=2000,
                    epochs=50,
                    validation_data=validation_generator,
                    validation_steps=800)

# save model
#model.save('saved_models/weights.best.inception.hdf5')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, None, None, 3 96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, None, None, 3 0           batch_normalization_283[0][0]    
__________________________________________________________________________________________________
conv2d_284

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/50
2000/2000 [==============================] - 335s 168ms/step - loss: 0.9745 - acc: 0.5065 - val_loss: 9.4043 - val_acc: 0.4133

Epoch 00001: val_loss improved from inf to 9.40432, saving model to saved_models/weights.best.inception.hdf5
Epoch 2/50
2000/2000 [==============================] - 312s 156ms/step - loss: 0.9201 - acc: 0.5299 - val_loss: 9.4045 - val_acc: 0.4133

Epoch 00002: val_loss did not improve
Epoch 3/50
2000/2000 [==============================] - 315s 157ms/step - loss: 0.9241 - acc: 0.5257 - val_loss: 9.4045 - val_acc: 0.4133

Epoch 00003: val_loss did not improve
Epoch 4/50
1269/2000 [==================>...........] - ETA: 1:20 - loss: 0.9182 - acc: 0.5324

KeyboardInterrupt: 

In [ ]:




# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)